## Run this notebook

You can launch this notebook using mybinder, by clicking the button below.

<a href="https://mybinder.org/v2/gh/NASA-IMPACT/veda-docs/HEAD?labpath=notebooks/datasets/air-quality-covid.ipynb">
<img src="https://mybinder.org/badge_logo.svg" alt="Binder" title="A cute binder" width="150"/> 
</a>

## Approach

   1. Identify available dates and temporal frequency of observations for a given collection - ODIAC CO2 emissions version 2022
   2. Pass the STAC item into raster API `/stac/tilejson.json` endpoint
   3. We'll visualize two tiles (side-by-side) allowing for comparison of each of the time points using `folium.plugins.DualMap`
   

## About the Data

The Open-Data Inventory for Anthropogenic Carbon dioxide (ODIAC) is a high-spatial resolution global emission data product of CO₂ emissions from fossil fuel combustion (Oda and Maksyutov, 2011). ODIAC pioneered the combined use of space-based nighttime light data and individual power plant emission/location profiles to estimate the global spatial extent of fossil fuel CO₂ emissions. With the innovative emission modeling approach, ODIAC achieved the fine picture of global fossil fuel CO₂ emissions at a 1x1km..

## The Case Study - ODIAC CO2 emissions 2022

In this notebook, we'll walk through the development of side-by-side monthly comparisons of CO2 levels for the different months (for example, 01-2000 and 01-2010)

## Querying the STAC API

In [1]:
import requests
from folium import Map, TileLayer

In [2]:
# Provide STAC and RASTER API endpoints
STAC_API_URL = "http://dev.ghg.center/api/stac"
RASTER_API_URL = "https://dev.ghg.center/api/raster"

# Name of the collection of interest - ODIAC
collection_name = "odiac-fossil-fuel-co2-emissions-2022"

In [3]:
collection = requests.get(f"{STAC_API_URL}/collections/{collection_name}").json()
collection

{'id': 'odiac-fossil-fuel-co2-emissions-2022',
 'type': 'Collection',
 'links': [{'rel': 'items',
   'type': 'application/geo+json',
   'href': 'https://gkynb1qvnl.execute-api.us-west-2.amazonaws.com/api/stac/collections/odiac-fossil-fuel-co2-emissions-2022/items'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://gkynb1qvnl.execute-api.us-west-2.amazonaws.com/api/stac/'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://gkynb1qvnl.execute-api.us-west-2.amazonaws.com/api/stac/'},
  {'rel': 'self',
   'type': 'application/json',
   'href': 'https://gkynb1qvnl.execute-api.us-west-2.amazonaws.com/api/stac/collections/odiac-fossil-fuel-co2-emissions-2022'}],
 'title': 'ODIAC Fossil Fuel CO₂ Emissions Version 2022',
 'assets': None,
 'extent': {'spatial': {'bbox': [[-180, -90, 180, 90]]},
  'temporal': {'interval': [['2000-01-01 00:00:00+00',
     '2021-12-01 00:00:00+00']]}},
 'license': 'CC-BY-4.0',
 'keywords': None,
 'providers': [{'url': 'https

Examining the contents of our `collection` under `summaries` we see that the data is available from January 2015 to December 2022. By looking at the `dashboard:time density` we observe that the periodic frequency of these observations is monthly. 

In [9]:
# Check total number of items available
items = requests.get(f"{STAC_API_URL}/collections/{collection_name}/items?limit=300").json()["features"]
print(f"Found {len(items)} items")

Found 264 items


This makes sense as there are 7 years between 2016 - 2022, with 12 months per year, meaning 84 records in total.  

Below, we'll use the `cog_default` values to provide our upper and lower bounds in `rescale_values`. 

In [10]:
rescale_values = {'max': 300, 'min': -10}

## Exploring Changes in Carbon Dioxide (CO₂) levels. Using the Raster API

We will explore changes in air quality due to changes in human behaviour resulting from the COVID-19 pandemic. With people largely confined to their homes to reduce the spread of the novel coronavirus, scientists were anticipated there were likely to be fewer cars, planes, and ships emitting fossil fuel pollution. In this notebook, we'll explore the impacts these government lockdowns had on specific air pollutants (i.e., NO₂) and explore these changes over time. We'll then visualize the outputs on a map using `folium`. 

In [14]:
# to access the year value from each item more easily, this will let us query more explicity by year and month (e.g., 2020-02)
items = {item["properties"]["start_datetime"][:7]: item for item in items} 

In [16]:
len(items)

264

Now we will pass the item id, collection name, and `rescaling_factor` to the `Raster API` endpoint. We will do this twice times, once for December 2021 and again for March 2021, so that we can visualize each event independently. 

In [36]:
december_2021_tile = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={items['2021-12']['collection']}&item={items['2021-12']['id']}"
    "&assets=co2_emissions"
    "&color_formula=gamma+r+1.05&colormap_name=rainbow"
    f"&rescale={rescale_values['min']},{rescale_values['max']}", 
).json()
december_2021_tile

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://2qncpyg24c.execute-api.us-west-2.amazonaws.com/api/raster/stac/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?collection=odiac-fossil-fuel-co2-emissions-2022&item=odiac-fossil-fuel-co2-emissions-2022-202112&assets=co2_emissions&color_formula=gamma+r+1.05&colormap_name=rainbow&rescale=-10%2C300'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-180.0, -90.0, 180.0, 90.0],
 'center': [0.0, 0.0, 0]}

In [37]:
march_2021_tile = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={items['2021-03']['collection']}&item={items['2021-03']['id']}"
    "&assets=co2_emissions"
    "&color_formula=gamma+r+1.05&colormap_name=cool"
    f"&rescale={rescale_values['min']},{rescale_values['max']}", 
).json()
march_2021_tile

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://2qncpyg24c.execute-api.us-west-2.amazonaws.com/api/raster/stac/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?collection=odiac-fossil-fuel-co2-emissions-2022&item=odiac-fossil-fuel-co2-emissions-2022-202103&assets=co2_emissions&color_formula=gamma+r+1.05&colormap_name=cool&rescale=-10%2C300'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-180.0, -90.0, 180.0, 90.0],
 'center': [0.0, 0.0, 0]}

## Visualizing CO2 emissions


In [38]:
# We'll import folium to map and folium.plugins to allow mapping side-by-side
import folium
import folium.plugins

# Set initial zoom and map for NO2 Layer
m = folium.plugins.DualMap(location=(33.6901, 118.9325), zoom_start=5)

# February 2020
map_layer_2020 = TileLayer(
    tiles=december_2021_tile["tiles"][1],
    attr="GHG",
    opacity=0.8,
)
map_layer_2020.add_to(m.m1)

# February 2022
map_layer_2022 = TileLayer(
    tiles=march_2021_tile["tiles"][1],
    attr="GHG",
    opacity=0.8,
)
map_layer_2022.add_to(m.m2)

m



In [40]:
december_2021_tile["tiles"][0]

'https://2qncpyg24c.execute-api.us-west-2.amazonaws.com/api/raster/stac/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?collection=odiac-fossil-fuel-co2-emissions-2022&item=odiac-fossil-fuel-co2-emissions-2022-202112&assets=co2_emissions&color_formula=gamma+r+1.05&colormap_name=rainbow&rescale=-10%2C300'

## Summary

In this case study we have successfully visualized how NASA monitors NO₂ emissions from space. By showcasing lockdown (February 2020) and post-lockdown (February 2022) snapshots of air quality side-by-side, we demonstrate how quickly atmospheric NO₂ responds to reductions in emissions and human behavior. 